# The Spectral Galerkin method
(with numerical integration)

<p style="margin-bottom:1cm;">

<center style="font-size:1.5em"> Approximates solution $u(x)$ using global basis functions $\phi_k(x)$

<img src="Chebyshev_Polynomials_of_the_First_Kind.svg" width="400">

$$
u(x) = \sum_{k=0}^{N-1}\hat{u}_k \phi_k(x)
$$



# Basis functions


| Family    | Basis (span)                             | Domain    |
|  :---:    |         :---:                            |   :---:   |
| Chebyshev | $$\{T_k\}_{k=0}^{N-1}$$                  | $$[-1, 1]$$ |
| Legendre  | $$\{L_k\}_{k=0}^{N-1}$$                  | $$[-1, 1]$$ |
| Fourier   | $$\{\exp(\text{i}kx)\}_{k=-N/2}^{N/2-1}$$| $$[0, 2\pi]$$ |
| Hermite   | $$\{H_k\}_{k=0}^{N-1}$$                  | $$[-\infty, \infty]$$|
| Laguerre  | $$\{La_k\}_{k=0}^{N-1}$$                 | $$[0, \infty]$$ |


# Bases with non-periodic boundary conditions

## Dirichlet

| family    | Basis (span)          | Boundary condition |
|-----------|-----------------------|----------|
| Chebyshev | $$\{T_k-T_{k+2}\}_{k=0}^{N-3}$$ | $$u(\pm 1) = 0$$ |
| Legendre  | $$\{L_k-L_{k+2}\}_{k=0}^{N-3}$$ | $$u(\pm 1) = 0$$ |
| Hermite   | $$\exp(-x^2)\{H_k\}_{k=0}^{N-1}$$ | $$u(\pm \infty) = 0$$ |
| Laguerre  | $$\exp(-x/2)\{La_k-La_{k+1}\}_{k=0}^{N-2}$$| $$u(0) = u(\infty) = 0$$ |

## Neumann $u'(\pm 1) = 0$

| family    | Basis (span)          |
|-----------|-----------------------|
| Chebyshev | $$\{T_k-\frac{k^2}{(k+2)^2}T_{k+2}\}_{k=0}^{N-3}$$ | 
| Legendre  | $$\{L_k-\frac{k(k+1)}{(k+2)(k+3)}L_{k+2}\}_{k=0}^{N-3}$$ |

## Biharmonic $u(\pm 1) = u'(\pm 1) = 0$

| family    | Basis (span)          |
|-----------|-----------------------|
| Chebyshev | $$\{T_k-\frac{2(k+2)}{k+3}T_{k+2}+\frac{k+1}{k+3} T_{k+4}\}_{k=0}^{N-5}$$ | 
| Legendre  | $$\{L_k-\frac{2(2k+5)}{(2k+7)}L_{k+2}+\frac{2k+3}{2k+7}L_{k+4}\}_{k=0}^{N-5}$$ |


# Multidimensional problems

$$
u(x, y) = \sum_{k=0}^{N_0-1}\sum_{l=0}^{N_1-1}\hat{u}_{kl} \phi_k(x)\psi_l(y)
$$

$$
u(x, y, z) = \sum_{k=0}^{N_0-1}\sum_{l=0}^{N_1-1} \sum_{m=0}^{N_2-1}\hat{u}_{klm} \phi_k(x)\psi_l(y)\xi_m(z)
$$

# Tensor product spaces

Use 1D bases, e.g.,
$$
\begin{align}
\phi_k(x) &= T_k(x)   & k \in \mathbf{k} = 0, 1, \ldots, N_0-1 \\
\psi_l(x) &= \exp(\text{i}lx) & l \in \mathbf{l} = 0, 1, \ldots, N_1-1 \\
\xi_m(x) &= L_m(x)-L_{m+2}(x) & m \in \mathbf{m} = 0, 1, \ldots, N_2-3
\end{align}
$$

Combine to create outer product bases with appropriate boundary conditions

$$
\begin{align}
V^{\mathbf{N}} &= \text{span}\{\phi_k(x)\psi_l(y) : (k, l) \in \mathbf{k} \times \mathbf{l} \} \\
W^{\mathbf{N}} &= \text{span}\{\phi_k(x)\psi_l(y)\xi_m(z) : (k, l, m) \in \mathbf{k} \times \mathbf{l} \times \mathbf{m} \}
\end{align}
$$

Mixing any 1D basis. Combine Legendre with Chebyshev if desired.

# Solving PDEs with the Spectral Galerkin method follows three steps


1. Choose global basis (test) functions satisfying the correct boundary conditions. Usually modified Chebyshev or Legendre polynomials, or Fourier exponentials

2. Transform PDEs to weak variational forms using the method of weighted residuals

3. Assemble and solve system of equations


# Weighted inner products

For the spectral Galerkin method the weighted inner product is defined as

$$
 (u, v)_w = \int_{\Omega} u \overline{v} w \, \mathbf{dx},
$$

where $w(\mathbf{x})$ is a weight associated with the chosen basis (different bases have different weights), and $v(\mathbf{x})$ and $u(\mathbf{x})$ are test and trial functions, respectively. The overline represents a complex conjugate.

Note that $\Omega$ can be any tensor product domain spanned by the chosen 1D bases. For a 2D basis with Chebyshev in both directions $\Omega=[-1, 1]^2$ and $\mathbf{dx}=dxdy$. For mixed Chebyshev with Fourier it is $\Omega=[-1, 1]\times[0, 2\pi]$. 


# Quadrature is used to approximate the integral

1D:
$$
\int_{-1}^1 u v w \, {dx} \approx \sum_{i=0}^{N-1} u(x_i) v(x_i) \omega_i,
$$

where $\{\omega_i\}_{i=0}^{N-1}$ are the quadrature weights associated with the chosen basis and quadrature rule. The associated quadrature points are denoted as $\{x_i\}_{i=0}^{N-1}$. For Chebyshev we can choose between `Chebyshev-Gauss` or `Chebyshev-Gauss-Lobatto`, whereas for Legendre the choices are `Legendre-Gauss` or `Legendre-Gauss-Lobatto`. 

2D:
$$
\int_{-1}^1\int_{0}^{2\pi} u v w \, {dxdy} \approx \sum_{i=0}^{N_0-1}\sum_{j=0}^{N_1-1} u(x_i, y_j) v(x_i, y_j) \omega^{(x)}_i \omega_j^{(y)} ,
$$


# Shenfun contains everything you need to solve PDEs with the Spectral Galerkin method

## Basic usage

Shenfun consists of classes and functions whoose purpose are to make it easier to implement PDE’s with the spectral Galerkin method in simple tensor product domains. The most important everyday tools are

* Basis
* TestFunction/TrialFunction
* inner
* project
* TensorProductSpace
* div/grad/curl

All are available after importing `shenfun`'s functionality

In [1]:
from shenfun import *

Create a Chebyshev basis $\{T_k\}_{k=0}^{N-1}$, $N\in \mathbb{Z}$

In [2]:
N = 10
C = Basis(N, 'Chebyshev')
x, w = C.points_and_weights()

Create a Legendre basis with Dirichlet boundary conditions $\{L_k - L_{k+2}\}_{k=0}^{N-3}$

In [3]:
L = Basis(N, 'Legendre', bc=(0, 0))

Create a `Function` $u(x) = \sum_{k=0}^{N-1} \hat{u}_k T_k(x)$

In [4]:
u = Function(C)
u[2] = 1

We know that $T_2(x) = 2x^2-1$. By setting $u(2)=1$ above, we fix the value of $\hat{u}_2$ and the `Function` `u` will evaluate to exactly $T_2(x)$. 

We can see this also by projecting the function $T_2$ to the basis $\{T_k\}_{k=0}^{N-1}$

In [5]:
import sympy as sp
x = sp.Symbol('x')
T2 = 2*x**2 - 1  # or simply T2 = sp.chebyshevt(2, x)
print(project(T2, C))
print(u)

[-4.44089210e-17  2.36612408e-16  1.00000000e+00  1.00665217e-17
 -8.49065980e-17 -3.14018492e-17 -3.42420318e-17 -2.41668305e-17
 -4.44089210e-17  8.02375870e-17]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


Create another Legendre basis and a tensorproductspace as the outer product of these bases

In [6]:
TP = TensorProductSpace(comm, (C, L))

In [7]:
u = Function(TP)
import matplotlib.pyplot as plt
X = TP.mesh()
plt.figure(figsize=(6, 6))
for x in X[0][:, 0]:
    plt.plot((x, x), (X[1][0, 0], X[1][0, -1]), 'k')
for y in X[1][0]:
    plt.plot((X[0][0, 0], X[0][-1, 0]) , (y, y), 'k')